# Загрузка библиотек

In [11]:
# Датасет с вопросами

!git clone https://github.com/vladislavneon/RuBQ.git

In [10]:
# Датасет Quora

!gdown --id 11Q2g2aA-n7fYK95XEB3d6m0CWKSt6rob
!unzip quora_question_pairs_rus.csv.zip

# Импорт библиотек

In [1]:
import fasttext
import requests
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
import nltk.data
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, RegexpTokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vaceslavefimov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Загрузка данных

In [17]:
filepath = "RuBQ/RuBQ_2.0/RuBQ_2.0_test.json"
with open(filepath, mode="r", encoding="utf-8") as file:
    lines = file.readlines()
    
questions = [result_json['question_text'] for result_json in json.loads(''.join(lines))]

In [18]:
quora_df = pd.read_csv('quora_question_pairs_rus.csv').drop('Unnamed: 0', axis=1)
quora_df

,question1,question2,is_duplicate
0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад",0
1,"как я могу увеличить скорость моего интернет-соединения, используя vpn",как повысить скорость интернета путем взлома через dns,0
2,"почему я мысленно очень одинок, как я могу это решить","найти остаток, когда математика 23 ^ 24 математика разделена на 24 23",0
3,которые растворяют в воде быстро сахарную соль метан и углеродный диоксид,какая рыба выживет в соленой воде,0
4,"астрология: я - луна-колпачок из козерога и крышка, поднимающая то, что это говорит обо мне","Я тройная луна-козерог и восхождение в козероге, что это говорит обо мне",1
...,...,...,...
404283,сколько ключевых слов есть на языке программирования ракетки последней версии,сколько ключевых слов есть в языке программирования perl в последней версии,0
404284,"ты веришь, что есть жизнь после смерти","правда ли, что есть жизнь после смерти",1
404285,что такое одна монета,что это за монета,0
404286,какова приблизительная годовая стоимость жизни во время учебы в uic chicago для индийского студента,"У меня проблема с волосами, но я хочу использовать продукт для укладки волос, который я должен предпочесть из гелевого воска и глины",0


# Обработка данных

In [22]:
def text_to_wordlist(text, remove_stopwords=False):
    text = re.sub(r"http[s]?://(?:[а-яА-Я]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", text)
    text = re.sub("[^а-яА-Я]"," ", text)
    words = text.lower().split()
    if remove_stopwords:
        stops = stopwords.words('russian')
        words = [w for w in words if not w in stops]
    return (words)


def process_text(text, tokenizer, remove_stopwords=False):
    raw_texts = tokenizer.tokenize(text.strip())
    texts = []
    for raw_text in raw_texts:
        if len(raw_text) > 0:
            texts.append(text_to_wordlist(raw_text, remove_stopwords))
    return texts

In [23]:
processed_texts = []
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')

texts = questions + quora_df.question1.tolist() + quora_df.question2.tolist()
texts = list(filter(lambda x: pd.notnull(x), texts))
for text in texts:
    processed_text = process_text(text, tokenizer)
    processed_texts += processed_text

In [24]:
filename = 'processed_texts.txt'

In [25]:
with open(filename, 'w') as f:
    for text in processed_texts:
        f.write(' '.join(text))
        f.write('\n')

# Обучение модели

In [26]:
fasttext_model = fasttext.train_unsupervised(filename, minn=3, maxn=4, dim=300) 

In [44]:
# построим эмбеддинг для каждого вопроса в датасете, удаляя при этом дубликаты

texts_df = pd.DataFrame(data={
    'text': texts,
    'embedding': [fasttext_model.get_sentence_vector(text) for text in texts]
}).drop_duplicates(subset=['text']).reset_index(drop=True)
texts_df.head(3)

,text,embedding
0,Что может вызвать цунами?,"[0.044315122, 0.010871048, 0.029821767, 0.038167942, 0.04884163, 0.0079813255, -0.09479028, 0.02214602, 0.006316959, -0.058758266, 0.032333843, -0.0072024176, 0.009663513, -0.013401447, 0.029670704, -0.015550475, -0.056715388, -0.0154038975, -0.013065408, 0.052290563, -0.013497455, 0.030696008, -0.032955877, 0.033623923, -0.039047383, 0.019412052, -0.028423443, 0.042598307, 0.00454716, 0.050415583, 0.0021588658, -0.022111196, 0.0069980863, 0.00010279007, -0.07667861, 0.030759275, -0.0656685, -0.023443563, 0.018622804, -0.061384156, 0.026389308, -0.015555304, 0.014877323, 0.015767371, -0.013030233, 0.0052310685, -0.048959836, 0.041098315, 0.0032106428, 0.047558945, 0.0053680884, 0.034276724, 0.0060699983, 0.01225781, -0.067310944, 0.040192675, 0.044395644, 0.0023803227, -0.0076669045, 0.034845404, 0.06317175, 0.025344271, -0.01943509, -0.040245853, -0.027217982, 0.00086563453, 0.041792925, 0.006640739, 0.014911151, 0.015458485, 0.013400449, 0.0012512356, -0.056235608, 0.0008115368, -0.039887104, 0.05389005, 0.020987386, 0.061558053, -0.006298762, -0.04639668, 0.025975592, -0.011680339, 0.06490635, 0.019210255, 0.01652912, -0.02303954, -0.0017986582, -0.0030795205, -0.05610976, 0.0025115516, 0.030918092, 0.011898616, -0.071306914, 0.014666501, 0.01946826, 0.0056292815, 0.013972516, 0.052689444, -0.07260863, 0.011781173, ...]"
1,Кто написал роман «Хижина дяди Тома»?,"[0.054791883, -0.011971297, -0.0011918049, 0.034816917, 0.02722923, 0.046566285, -0.02803547, -0.025177103, -0.047746986, -0.060767204, -0.006137914, 0.017785266, 0.030672556, 0.017952427, 0.045217574, 0.045413654, -0.036938068, -0.034237493, -0.014417522, 0.015652684, 0.0021483365, -0.008443579, -0.044306453, 0.025103807, 0.0019285984, 0.027650926, -0.048009038, 0.03399605, -0.014769684, 0.02954333, -0.0305438, -0.0018267035, 0.0060064383, -0.03492912, -0.04377044, -0.005100323, -0.05109803, -0.11748327, 0.01689487, -0.02691999, 0.015887683, -0.046000794, 0.0011281839, -0.023847898, -0.0085701, -0.02577728, 0.004134213, 0.041348383, 0.033210352, 0.03194347, 0.04400214, -0.008842265, 0.04693842, 0.042871162, 0.016305175, 0.006519884, 0.12749869, 0.045726225, -0.004073342, 0.027053706, -0.02302128, -0.0029018435, -0.005570153, -0.010238489, -0.028535439, -0.010140242, -0.0005358122, 0.013704864, 0.036002975, 0.035605293, -0.022414481, 0.032329552, -0.0011849962, -0.0071214414, -0.05809952, 0.024325725, -0.009504844, 0.07259759, -0.059243064, 0.004258436, -0.055889986, 0.018517233, 0.010286246, 0.011530748, -0.028741416, -0.029688355, -0.016866717, -0.0030694408, -0.047002994, -0.023142135, 0.04943667, 0.02025037, -0.02595719, -0.017810512, 0.024402224, -0.03752614, 0.032384086, -0.0080803605, -0.009230387, 0.009496856, ...]"
2,Кто автор пьесы «Ромео и Джульетта»?,"[0.030386446, 0.028907573, 0.006990945, -0.013711916, 0.036098644, 0.004863815, -0.021151034, 0.019961808, -0.032302085, 0.0037019576, -0.0126191005, 0.004338437, 0.030118696, 0.00981353, 0.047025546, -0.0032756757, -0.037378944, -0.03056981, -0.008256078, 0.0081425235, -0.00864208, -0.031948626, -0.04869178, 0.012475077, 0.003766106, -0.014534399, -0.030114636, 0.015575975, 0.026563099, 0.048832722, -0.023145054, -0.0061658267, -0.04191442, -0.009242686, -0.06894898, -0.029587831, -0.030182224, 0.0004913292, 0.0029695488, -0.02248016, 0.009861695, -0.012783274, 0.03820328, 0.0038620732, -0.006158007, -0.0421881, 0.008089723, 0.049528398, -0.00704574, 0.06870546, -0.016709255, 0.009366987, 0.014327071, 0.034447845, 0.024398334, 0.019776355, 0.07804814, 0.028701073, -0.03049243, -0.022291735, -0.020286724, 0.025654819, -0.01834745, -0.033386774, -0.0011138264, 0.006274735, -0.01189637, -0.03112146, 0.0032391835, -0.0020507611, 0.01578553, -0.009209137, -0.033469513, -0.023051307, -0.021797542, -0.050341688, -0.026488734, 0.024350535, -0.028967049, 0.04017462, 0.0023731168, -0.032063574, -0.014715924, 0.017737053, 0.020690208, -0.038081642, 9.660163e-05, 0.010691639

In [28]:
def cosine(u, v):
    res = np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
    return res

In [29]:
# также добавим поддержку исправления опечаток слов с помощью Yandex Spellchecker API

def check_spelling(text):
    domain = "https://speller.yandex.net/services/spellservice.json"
    words = text.split()
    if len(words) == 1:
        request = requests.get(domain + "/checkText?text=" + words[0])
        if request.json():
            return request.json()[0]["word"], request.json()[0]["s"]
        else:
            return None
    
    elif len(words) > 1:
        words = "+".join(words)
        request = requests.get(domain + "/checkText?text=" + words)
        if request.json():
            response = [(i["word"], i["s"]) for i in request.json()]
            return response
        else:
            return None
    return None

In [45]:
def most_similar_texts(text, n=10, verbose=True):
    
    edited_words = []
    words = text.split()
    for word in words:
        edited_word = check_spelling(word)
        edited_words.append(word if edited_word is None else edited_word[1][0])
    edited_text = ' '.join(edited_words)
    
    if verbose:
        print(f'Исправленный текст: {edited_text}')
    
    embedded_text = fasttext_model.get_sentence_vector(edited_text)
    cosines = [cosine(x, embedded_text) for x in texts_df['embedding']]
    argpartition = np.argpartition(cosines, -n)[-n:]
    
    return pd.DataFrame(data={
        'text': [texts_df.loc[i, 'text'] for i in argpartition],
        'cosine': [cosines[i] for i in argpartition]
    }).sort_values(by='cosine', ascending=False).reset_index(drop=True)

# Вывод наиболее похожих вопросов

In [31]:
text = 'Какой город самый большой в мире?'
most_similar_texts(text)

Исправленный текст: Какой город самый большой в мире?


,text,cosine
0,какой самый большой город в мире,0.957669
1,какой самый большой город в мире по площади,0.904160
2,какой самый большой город в Финляндии?,0.903352
3,какой самый большой океан в мире,0.903096
4,какой самый большой город,0.883355
5,самый большой хит-фильм в мире,0.870162
6,самый большой мошенник в мире,0.869815
7,самый красивый город в мире,0.865701
8,какой величайший город в мире,0.863240
9,самый большой грузовик в мире,0.858967


In [32]:
text = 'Какой вуз самый лучший?'
most_similar_texts(text)

Исправленный текст: Какой вуз самый лучший?


,text,cosine
0,самый лучший самый крутой кошелек,0.825918
1,какой самый быстрый способ стать лучше,0.821844
2,какой лучший онлайн-университет,0.815515
3,Какой лучший онлайн-институт,0.812724
4,какой самый лучший древовидный альбом,0.809761
5,какой самый быстрый способ стать ведьмой,0.809025
6,какой лучший университет,0.808458
7,какой самый быстрый способ обучения,0.803648
8,какой лучший и самый простой способ снять стресс,0.801913
9,какой самый быстрый способ учиться,0.801223


In [33]:
text = 'Какой драматург написал пьесу «Ромео и Джульетта»'
most_similar_texts(text)

Исправленный текст: Какой драматург написал пьесу «Ромео и Джульетта»


,text,cosine
0,Кто автор пьесы «Ромео и Джульетта»?,0.836746
1,Какую оперу на сюжет «Ромео и Джульетты» написал композитор Винченцо Беллини?,0.782387
2,какой ваш любимый тип музыки хип-хоп и рэп б хип-хоп и рок с твердым металлом и скалой,0.752977
3,"Какой композитор написал музыку к кинофильмам «Семнадцать мгновений весны» и «Ирония судьбы, или С лёгким паром!»?",0.748965
4,"композитор камерной музыки, который написал пьесу для родной американской флейты, сказал, что все на флейтисты звучат одинаково и не являются настоящими музыкантами, это правда",0.740513
5,кто-нибудь может написать лирику и перевести песню lekha из драмы танца кучипуди бхама калапам,0.728612
6,"Кто является режиссером и комедии «Дети Дон Кихота», и военной драмы «Служили два товарища», и приключенческого фильма «Два капитана»?",0.727106
7,Каковы некоторые примеры любви в романе и Джульет по Уильяму Шекспиру,0.723831
8,что случилось с лео в книжной серии: перси Джексон и герои олимпуса,0.720954
9,который написал рамаяну и махабхарату,0.719697


In [34]:
text = 'Какие причины у глобального потепления?'
most_similar_texts(text)

Исправленный текст: Какие причины у глобального потепления?


,text,cosine
0,каковы причины глобального потепления,0.850946
1,каковы основные причины глобального потепления,0.825273
2,какие последствия глобального потепления,0.808981
3,какие страны выиграют от глобального потепления,0.792608
4,каковы экологические последствия глобального потепления,0.779185
5,каковы некоторые важные причины и следствия в отношении глобального потепления,0.767071
6,каковы потенциальные социальные последствия глобального потепления,0.763321
7,есть ли у нас хорошие способы остановить глобальное потепление,0.761330
8,каковы нынешние усилия по предотвращению глобального потепления,0.759412
9,каковы положительные эффекты глобального потепления,0.753763


In [35]:
text = 'Какой сериал можно посмотреть на выходных?'
most_similar_texts(text)

Исправленный текст: Какой сериал можно посмотреть на выходных?


,text,cosine
0,можно посмотреть розовый фильм с семьей,0.747318
1,"Бэтмен: анимированная серия не на какой-либо сети кабельного телевидения или netflix, почему так трудно найти, где я могу законно ее посмотреть",0.742690
2,как можно как можно скорее посмотреть сериал 5-го сезона sherlock 2 лежащий детектив в Индии онлайн,0.742529
3,"где я могу посмотреть драконский бал супер английский, дублированный на телевизоре",0.738261
4,"что такое ужасный фильм, который можно посмотреть на Хэллоуин",0.736013
5,где можно посмотреть на английском dub boruto Наруто фильм,0.732582
6,где я могу скачать или посмотреть сериал серии krishi,0.727529
7,"сколько нравится на facebook, вам нужно зарабатывать деньги, то же самое можно посмотреть на просмотрах youtube",0.726659
8,возможно ли смотреть 3D-фильм на окулусной рифтовой гарнитуре,0.725345
9,"лучше смотреть на фурго, когда сериал видел фильм первым или нет",0.724886


In [36]:
text = 'Кокой город самй бальшой?'
most_similar_texts(text)

Исправленный текст: Какой город самый большой


,text,cosine
0,какой самый большой город,0.950934
1,какой самый большой город в мире,0.863766
2,какой самый большой город в Финляндии?,0.856921
3,какой самый большой секрет,0.851224
4,какой парень самый большой поворот,0.829610
5,Какой океан самый маленький?,0.820790
6,какой самый большой секрет Индии,0.818496
7,какой самый большой океан в мире,0.805271
8,какой самый красивый город в Новой Англии,0.802319
9,какой самый большой круг трафика,0.801144


In [37]:
text = 'Как можно научыться еграть в шашматы?'
most_similar_texts(text)

Исправленный текст: Как можно научиться играть в шахматы


,text,cosine
0,как можно научиться играть в шахматы?,0.941079
1,как научиться играть в шахматы,0.906011
2,как научиться играть в шахматы?,0.901935
3,как я могу научиться играть в шахматы,0.863571
4,как играть в шахматы,0.852608
5,почему я должен учиться играть в шахматы,0.850402
6,как вы научились играть в шахматы,0.840919
7,как я научиться играть в футбол,0.839214
8,как я могу научиться играть в шахматы в своем уме,0.836878
9,как не играть в шахматы,0.833575


In [38]:
text = 'Как улучжать ангийций язик?'
most_similar_texts(text)

Исправленный текст: Как улучшать английский язык


,text,cosine
0,как улучшить английский язык,0.869644
1,как улучшить свой английский язык,0.835849
2,официальный английский язык,0.817735
3,как я улучшу свой английский язык,0.806038
4,как улучшить английский язык в неанглийскоязычной среде,0.802391
5,который лучше изучать британский английский или английский английский,0.801881
6,который лучше изучать английский английский или британский английский,0.801881
7,английский язык - это рабский язык,0.800317
8,как возник английский язык,0.796975
9,как улучшить свой общий родной английский язык,0.794886


In [39]:
text = 'Как я магу зстатъ сомым умным?'
most_similar_texts(text)

Исправленный текст: Как я магу зстатъ самым умным?


,text,cosine
0,как я могу стать очень умным,0.778646
1,как я могу стать умным и умным,0.771861
2,как я должен стать умным,0.766701
3,как я могу стать умным,0.762504
4,как я могу быть самым умным человеком,0.757890
5,как я могу стать умным уличным,0.754845
6,Как я могу стать порнозвездой,0.750193
7,как я становлюсь самым умным парнем в комнате,0.750174
8,как я могу быть самым умным человеком когда-либо,0.748055
9,как я становлюсь умным и умным,0.746328


In [40]:
text = 'Что нжно сделодь, чтобы вставтъ утром?'
most_similar_texts(text)

Исправленный текст: Что нужно сделать чтобы вставать утром?


,text,cosine
0,"что я должен сделать, чтобы встать рано утром",0.862533
1,"что я должен сделать, чтобы проснуться рано утром?",0.843579
2,"что нужно сделать, чтобы сделать кого-то счастливым",0.837676
3,"что я могу сделать, чтобы встать рано",0.828847
4,"что я могу сделать, чтобы быстрее проснуться утром",0.817405
5,"это лучшее упражнение, которое нужно сделать утром",0.816671
6,"что нужно сделать, чтобы сделать правду или посмеяться",0.812472
7,"что нужно сделать, чтобы похудеть",0.809616
8,"что мы можем сделать, чтобы проснуться утром, не уставая",0.807657
9,"что я должен сделать, чтобы почувствовать свежесть утром",0.806936


In [41]:
text = 'Как нойти в себе уввереност?'
most_similar_texts(text)

Исправленный текст: Как найти в себе уверенность


,text,cosine
0,как повысить уверенность в себе,0.831675
1,как я могу вернуть себе уверенность в себе,0.826987
2,как восстановить свою уверенность в себе,0.821999
3,каковы способы вернуть себе уверенность в себе,0.811507
4,как я могу обрести уверенность в себе,0.801999
5,как я развиваю уверенность в себе,0.800211
6,как я могу вернуть утраченную уверенность в себе,0.798213
7,имеет уродливую подругу часть низкой уверенности в себе,0.796595
8,"я теряю уверенность в себе и мотивацию, как я могу построить свою уверенность в себе и мотивацию",0.794607
9,как я могу укрепить уверенность в себе,0.791885


In [42]:
text = 'Могу ли я зоробатыват в покерр?'
most_similar_texts(text)

Исправленный текст: Могу ли я зарабатывать в покер


,text,cosine
0,могу ли я зарабатывать тысячи в месяц в покере?,0.923776
1,могу ли я зарабатывать деньги в офлайн-приложении?,0.916019
2,"могу ли я зарабатывать 10 000 в месяц, играя в онлайн-покер как хастлер",0.907385
3,"могу ли я зарабатывать на реальные деньги, играя в покемон",0.907127
4,"могу ли я зарабатывать 700 долларов в день, играя в покер",0.905574
5,могу ли я зарабатывать деньги на покере,0.902776
6,"могу ли я зарабатывать 40 000 в месяц, играя в покер",0.901518
7,"могу ли я зарабатывать 800000 в год, играя в онлайн-покер",0.901230
8,"могу ли я зарабатывать 50 000 в месяц, играя в покер",0.901204
9,"могу ли я зарабатывать 60 000 в месяц, играя в покер",0.900370


# Оценка качества
Для оценки качества возьмем случайные 100 пар вопросов из датасета Quora. Для каждой пары будем брать первый вопрос и искать для него 1000 наиболее подходящих вопросов. Если среди этих 1000 вопросов будет второй вопрос, то будет считать, что запрос отработал корректно, иначе - нет.

**NB:** Безусловно, для более точной оценки качества следует брать большее количество пар вопросов, но в целях экономии времени вычислений в качестве демонстрации было решено взять только 100 вопросов.

In [80]:
test_df = quora_df.query('is_duplicate == 1').sample(100)
correct_answers = wrong_answers = 0

for question1, question2 in zip(test_df['question1'], test_df['question2']):
    if question2 in most_similar_texts(question1, n=1000, verbose=False)['text'].values:
        correct_answers += 1
    else:
        wrong_answers += 1

In [81]:
accuracy = correct_answers / (correct_answers + wrong_answers)
print(f'Доля правильных ответов: {accuracy}')

Доля правильных ответов: 0.68
